In [ ]:
import pandas as pd
from dataclasses import dataclass
from torch.utils.data import DataLoader
from typing import List
import pytorch_lightning as pl
import torch
from torch import nn

import torch.nn.functional as F
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt

from utils import TimeseriesDataset
from losses import asymetric_gaussian_nll_loss

In [ ]:
@dataclass
class Configs:
    csv_path : str = "datasets/synthetic_sin_w_asym_noise.csv"
    model_output_path: str = "runs/asymetric_gaussian_rnn"
    features = ['sin']
    targets = ['target']
    input_length: int = 10
    learning_rate: float = 1e-3
    weight_decay: float = 1e-4
    epochs: int = 50
    early_stopping: int = 4
    batch_size: int = 32
    lstm_hidden_size: int = 32
    lstm_num_layers: int = 1
    device: str = "cuda:0"

In [ ]:
dataset = pd.read_csv(Configs.csv_path)
dataset = dataset.set_index("date")

In [ ]:
train_dataset = TimeseriesDataset(dataset, 
                                  features=Configs.features,
                                  targets=Configs.targets,
                                  input_length=Configs.input_length,
                                  start_date="1980-01",
                                  end_date="1990-01",
                                  scaler=None)

val_dataset = TimeseriesDataset(dataset, 
                                features=Configs.features,
                                targets=Configs.targets,
                                input_length=Configs.input_length,
                                start_date="1990-01",
                                end_date="2000-01",
                                scaler=train_dataset.scaler)

train_loader = DataLoader(train_dataset,
                          batch_size=Configs.batch_size,
                          shuffle=True,
                          num_workers=0)

val_loader = DataLoader(val_dataset,
                        batch_size=Configs.batch_size,
                        shuffle=False,
                        num_workers=0)

In [ ]:
class AsymmetricGaussianLSTM(pl.LightningModule):
    def __init__(
        self,
        features: List[str] = ['col'],
        targets: List[str] = ['col'],
        lstm_hidden_size: int = 32,
        lstm_num_layers: int = 1,
        learning_rate: float = 1e-3,
        weight_decay: float = 1e-4,
    ):

        super().__init__()
        self.save_hyperparameters()
        print(f"Init model, hparams:\n{self.hparams}\n")
        self.lstm_encoder = nn.GRU(
            len(features), lstm_hidden_size, lstm_num_layers, batch_first=True
        )
        self.mu = nn.Linear(lstm_hidden_size, len(targets))
        self.sigma_left = nn.Linear(lstm_hidden_size, len(targets))
        self.sigma_right = nn.Linear(lstm_hidden_size, len(targets))

    def forward(self, x):
        h, _ = self.lstm_encoder(x)
        h = h[:, -1, :]  # last rnn hidden state
        mu = self.mu(h)
        sigma_left = self.sigma_left(h)
        sigma_left = F.elu(sigma_left) + 1.0 + 1e-15
        sigma_right = self.sigma_right(h)
        sigma_right = F.elu(sigma_right) + 1.0 + 1e-15
        return mu, sigma_left, sigma_right

    def configure_optimizers(self):
        return torch.optim.Adam(
            self.parameters(),
            self.hparams.learning_rate,
            weight_decay=self.hparams.weight_decay,
        )

    def _step(self, batch, log_mode: str = "train"):
        """ """
        x, y = batch[0], batch[1]
        x = x.to(dtype=torch.float32, device=self.device)
        y = y.to(dtype=torch.float32, device=self.device)
        mu, sigma_left, sigma_right = self(x)
        loss = asymetric_gaussian_nll_loss(mu, y, sigma_left, sigma_right)
        # loss = F.mse_loss(mu, y)
        self.log(f"{log_mode}_loss", loss)
        return loss

    def training_step(self, batch, batch_idx):
        return self._step(batch, log_mode="train")

    def validation_step(self, batch, batch_idx):
        return self._step(batch, log_mode="val")

    def predict(self, data_loader: torch.utils.data.DataLoader):
        """Predict function for inference/testing the model

        Args:
            data_loader (torch.utils.data.DataLoader): You should pass the test loader here

        Returns:
            tuple(np.arr, np.arr): predictions, ground_truth
        """
        self.eval()
        result_dfs = []
        for x, y, date in data_loader:
            with torch.no_grad():
                x = x.to(dtype=torch.float32, device=self.device)
                y = y.to(dtype=torch.float32, device=self.device)
                mu, sigma_left, sigma_right = self(x)
                mu = mu.detach().cpu().numpy()
                sigma_left = sigma_left.detach().cpu().numpy()
                sigma_right = sigma_right.detach().cpu().numpy()
                y = y.detach().cpu().numpy()
                df_y = pd.DataFrame(index=date, data=y, columns=[f"y_{c}" for c in self.hparams.targets])
                df_mu = pd.DataFrame(index=date, data=mu, columns=[f"mu_{c}" for c in self.hparams.targets])
                df_sigma_left = pd.DataFrame(index=date, data=sigma_left, columns=[f"sigma_left_{c}" for c in self.hparams.targets])
                df_sigma_right = pd.DataFrame(index=date, data=sigma_right, columns=[f"sigma_right_{c}" for c in self.hparams.targets])
                df_res = pd.concat([df_mu, df_y, df_sigma_left, df_sigma_right], axis=1)
                result_dfs.append(df_res)
        return pd.concat(result_dfs)


In [ ]:
model = AsymmetricGaussianLSTM(
    features=Configs.features,
    targets=Configs.targets,
    learning_rate=Configs.learning_rate,
    weight_decay=Configs.weight_decay,
    lstm_hidden_size=Configs.lstm_hidden_size,
    lstm_num_layers=Configs.lstm_num_layers
    )
model.to(Configs.device)

In [ ]:
early_stop_callback = EarlyStopping(
   monitor='val_loss',
   patience=Configs.early_stopping,
   verbose=True
)
checkpoint_callback = ModelCheckpoint(monitor="val_loss", save_weights_only=True)

if Configs.device == "cpu":
    trainer = pl.Trainer(default_root_dir=Configs.model_output_path,
                         max_epochs=Configs.epochs,
                         callbacks=[checkpoint_callback, early_stop_callback])
else:
    device_name, device_id = Configs.device.split(":")  
    assert device_name == "cuda", "Configs>device should be cpu or cuda"  
    trainer = pl.Trainer(default_root_dir=Configs.model_output_path,
                         max_epochs=Configs.epochs,
                         callbacks=[checkpoint_callback, early_stop_callback],
                         gpus=device_id)

trainer.fit(model, 
            train_dataloader=train_loader, 
            val_dataloaders=val_loader)

In [ ]:
model = AsymmetricGaussianLSTM.load_from_checkpoint(trainer.checkpoint_callback.best_model_path)
model.to(Configs.device)

In [ ]:
test_dataset = TimeseriesDataset(dataset, 
                                features=Configs.features,
                                targets=Configs.targets,
                                input_length=Configs.input_length,
                                start_date="2020-01",
                                end_date="2021-02",
                                scaler=train_dataset.scaler)

test_loader = DataLoader(test_dataset,
                        batch_size=Configs.batch_size,
                        shuffle=False,
                        num_workers=0)

In [ ]:
df_preds = model.predict(test_loader)
df_preds

In [ ]:
fig, ax = plt.subplots(figsize=(15,15))

y = df_preds["y_target"]
mu = df_preds["mu_target"]
sigma_left = df_preds["sigma_left_target"]
sigma_right = df_preds["sigma_right_target"]

ax.plot(df_preds.index, y, '-', color="blue")
ax.plot(df_preds.index, mu, '-', color="red")
ax.fill_between(df_preds.index, 
                mu - 3 * sigma_left, 
                mu + 3 * sigma_right, 
                alpha=0.8)
